In [7]:
import psycopg2
import pandas as pd
from datetime import datetime, date

In [101]:
conn = psycopg2.connect(database="accountdb", user="chiubj", 
                        password="bunnygood", host="128.110.25.99", 
                        port="5432")
cur  = conn.cursor()


In [104]:
orders = pd.read_csv('status/orders.csv')
orders['委託日期'] = pd.to_datetime(orders['委託日期'])
trades = pd.read_csv('status/trades.csv')
trades['成交日期'] = pd.to_datetime(trades['成交日期'])

In [102]:

for order_index in range(len(orders)):
    if date(datetime.now().year, datetime.now().month, datetime.now().day) == date(orders.iloc[order_index]['委託日期'].year, orders.iloc[order_index]['委託日期'].month, orders.iloc[order_index]['委託日期'].day):
        record = [str(orders.iloc[order_index]['交易員代碼']), orders.iloc[order_index]['委託書號'], orders.iloc[order_index]['交易別'], orders.iloc[order_index]['委託日期'], orders.iloc[order_index]['委託時間'],
        str(orders.iloc[order_index]['代碼']), orders.iloc[order_index]['委託種類'], orders.iloc[order_index]['方向'], int(orders.iloc[order_index]['委託部位']), 
        float(orders.iloc[order_index]['委託價格']), orders.iloc[order_index]['狀態'], str(orders.iloc[order_index]['策略別'])]

        table_columns = '(trader_id, order_id, security_type, order_date, order_time, code, order_type, action, order_qty, order_price, status, strategy)'
        postgres_insert_query = f"""INSERT INTO accountdb.dealer.orders {table_columns} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
        cur.execute(postgres_insert_query, record)

        conn.commit()


In [113]:

for trade_index in range(len(trades)):
    if date(datetime.now().year, datetime.now().month, datetime.now().day) == date(trades.iloc[trade_index]['成交日期'].year, trades.iloc[trade_index]['成交日期'].month, trades.iloc[trade_index]['成交日期'].day):
        record = [str(trades.iloc[trade_index]['交易員代碼']), trades.iloc[trade_index]['委託書號'], trades.iloc[trade_index]['交易別'], trades.iloc[trade_index]['成交日期'], trades.iloc[trade_index]['成交時間'],
        str(trades.iloc[trade_index]['代碼']), trades.iloc[trade_index]['委託種類'], trades.iloc[trade_index]['方向'], int(trades.iloc[trade_index]['成交部位']), 
        float(trades.iloc[trade_index]['成交價格']), trades.iloc[trade_index]['狀態'], str(trades.iloc[trade_index]['策略別']), trades.iloc[trade_index]['成交序號 (seqno)']]

        table_columns = '(trader_id, order_id, security_type, trade_date, trade_time, code, order_type, action, qty, price, status, strategy, seqno)'
        postgres_insert_query = f"""INSERT INTO accountdb.dealer.trades {table_columns} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
        cur.execute(postgres_insert_query, record)

        conn.commit()


In [5]:
def postgre_query(query):
    conn = psycopg2.connect(database="accountdb", user="chiubj", 
                        password="bunnygood", host="128.110.25.99", 
                        port="5432")
    cur  = conn.cursor()

    cur.execute(query)
    conn.commit()
    cur.close()
    conn.close()

In [145]:
postgre_query("ALTER TABLE accountdb.dealer.trades ADD order_type char(3);")

In [9]:
postgre_query("ALTER TABLE accountdb.dealer.strategy ADD order_low_ratio double precision;")

In [125]:
table_column = 'code'
postgres_delete_query = f"""DELETE FROM accountdb.dealer.orders where {table_column} = %s;"""
delete_params = ('6414',)
cur.execute(postgres_delete_query, delete_params)
conn.commit()

In [131]:
import json

with open('./settings/strategy_info.json', "r") as f:
    strategy_info = json.load(f)
    

In [132]:
strategy_info

{'法說會前主力蠢蠢欲動': {'Leverage_Ratio': 0.64,
  'Expected_MDD': 54.0,
  'Expected_Daily_Return': 18.74},
 '注意股10日多': {'Leverage_Ratio': 0.34,
  'Expected_MDD': 71.5,
  'Expected_Daily_Return': 32.6},
 '營收首次創新高': {'Leverage_Ratio': 0.17,
  'Expected_MDD': 99.04,
  'Expected_Daily_Return': -11.21},
 '籌碼雷達5日多': {'Leverage_Ratio': 0.57,
  'Expected_MDD': 55.96,
  'Expected_Daily_Return': -18.86},
 '處置股10日多': {'Leverage_Ratio': 0.4,
  'Expected_MDD': 59.06,
  'Expected_Daily_Return': 60.12},
 '開高動能強飆股_1': {'Leverage_Ratio': 0.18,
  'Expected_MDD': 73.43,
  'Expected_Daily_Return': 23.82}}

In [137]:

for strategy in strategy_info:
    postgre_query(f"UPDATE accountdb.dealer.strategy SET Leverage_Ratio = {strategy_info[strategy]['Leverage_Ratio']} WHERE Name = '{strategy}';")
    postgre_query(f"UPDATE accountdb.dealer.strategy SET Expected_MDD = {strategy_info[strategy]['Expected_MDD']} WHERE Name = '{strategy}';")
    postgre_query(f"UPDATE accountdb.dealer.strategy SET Expected_Daily_Return = {strategy_info[strategy]['Expected_Daily_Return']} WHERE Name = '{strategy}';")


In [53]:
postgre_query(f"UPDATE accountdb.dealer.strategy SET Leverage_Ratio = {strategy_info[strategy]['Leverage_Ratio']} WHERE Name = '{strategy}';")

In [49]:
postgre_query(f"UPDATE accountdb.dealer.strategy SET Leverage_Ratio = {strategy_info[strategy]['Leverage_Ratio']} WHERE na = 1;")

In [42]:
for strategy in strategy_info:
    print(strategy)

法說會前主力蠢蠢欲動
注意股10日多
營收首次創新高
籌碼雷達5日多
處置股10日多
開高動能強飆股_1


In [60]:
from tsdb_client import TSDBClient

In [61]:
cli = TSDBClient(
    host="128.110.25.99",
    port=5432,
    user="chiubj",
    password="bunnygood",
    db="accountdb"
)

In [11]:
# position
cli.execute_query("""
    select 
        strategy, 
        code,
        sum((case when action = 'B' then 1 else -1 end) * qty) as position,
        sum(price*qty) / sum(qty) as avg_price
    from dealer.trades
    group by strategy, code
    """, 
    out_type='df'
)

,strategy,code,position,avg_price
0,1,2412,5,126.5
1,1,6414,2,257.5


In [54]:
# profit and loss
df = cli.execute_query("""

    select t_st.name as 策略, 
        t_pos.code as 代碼,
        t_pos.position as 部位,
        t_pos.avg_price as 成本價,
        t_quote.close as 收盤價,
        ROUND(position * (close - avg_price))*1000 as 損益 into strategy_pl
    from (
        select 
            strategy, 
            code,
            sum((case when action = 'B' then 1 else -1 end) * qty) as position,
            sum(price*qty) / sum(qty) as avg_price
        from dealer.trades
        group by strategy, code
    )t_pos
    left join public.quote_snapshots t_quote on t_pos.code = t_quote.code
    left join dealer.strategy t_st on t_pos.strategy = t_st.id 

    """
)

Error: relation "strategy_pl" already exists



In [70]:
cli.execute_query('''

create TEMP TABLE tb_trade
( 
	SN int --流水編號
	,PID varchar(10) --商品代碼
	,TranDate date --交易日期
	,TranCode char(3) --交易種類 
	,Qty int
	,Price decimal(10,3)
	,CurrentQty int  --累計庫存(檢核用)
	,CurrentValue decimal(10,3) --累計成本(檢核用)
)
	

''')

0

In [44]:
# profit and loss
df = cli.execute_query("""
    select *,
    sum(price_sold - price_purchased) over(order by rn) as profit 
from
    (
        select 
            row_number() over(order by trade_date, code) as rn,
            (case when action = 'B' then 1 else -1 end) * qty as position,
            code, price as price_purchased
        from dealer.trades, generate_series(1, abs(qty))
        where (case when action = 'B' then 1 else -1 end) > 0
    ) p
    full join
    (
        select
            row_number() over(order by trade_date, code) as rn,
            ((case when action = 'B' then 1 else -1 end) * qty) as position,
            code, price as price_sold
        from dealer.trades, generate_series(1, abs(qty))
        where (case when action = 'B' then 1 else -1 end) < 0
    ) s using (rn, code)
    
    """, 
    out_type='df'
)